## IMPORTS

In [0]:
! pip install syft

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 41.7MB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
     |████████████████████████████████| 389kB 41.8MB/s 
     |████████████████████████████████| 450kB 31.4MB/s 
     |████████████████████████████████| 71kB 24.0MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
     |████████████████████████████████| 51kB 19.4MB/s 
     |████████████████████████████████| 122kB 39.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [0]:
import torch
from torch import nn
from torch import optim
import syft as sy
hook = sy.TorchHook(torch)

W0619 14:43:55.638832 140391203641216 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0-rc1). Fix this by compiling custom ops.
W0619 14:43:55.747333 140391203641216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



## FEDERATED LEARNING

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = torch.tensor([1,2,3,4])
y = torch.tensor([1,1,1,1])

In [0]:
x_ptr = x.send(bob)

In [0]:
y-x_ptr

TensorsNotCollocatedException: ignored

In [0]:
y_ptr = y.send(alice)

In [0]:
alice._objects

{474167764: tensor([1, 1, 1, 1])}

In [0]:
y = y_ptr.get()

In [0]:
y

tensor([1, 1, 1, 1])

In [0]:
alice._objects

{}

In [0]:
x = x_ptr.get()

In [0]:
x

tensor([1, 2, 3, 4])

In [0]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z

(Wrapper)>[PointerTensor | me:4889677902 -> bob:71776085183]

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:10927137073 -> bob:51145077860]

In [0]:
z

(Wrapper)>[PointerTensor | me:4889677902 -> bob:71776085183]

In [0]:
x.grad

(Wrapper)>[PointerTensor | me:43178543057 -> bob:95245649495]::grad

In [0]:
x_ = x.get()

In [0]:
x_

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x_.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
y_ = y.get()

In [0]:
y_.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
bob._objects

{71776085183: tensor(20., grad_fn=<SumBackward0>)}

In [0]:
z = z.get()

In [0]:
z

tensor(20., requires_grad=True)

In [0]:
bob._objects

{}

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)
        #print(pred)
        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)


In [0]:
train()

tensor(1.6215)
tensor(0.5663)
tensor(0.3284)
tensor(0.2087)
tensor(0.1343)
tensor(0.0867)
tensor(0.0560)
tensor(0.0363)
tensor(0.0236)
tensor(0.0153)
tensor(0.0100)
tensor(0.0065)
tensor(0.0043)
tensor(0.0028)
tensor(0.0019)
tensor(0.0013)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)


In [0]:
model(torch.tensor([1,1.],requires_grad=True))

tensor([0.9805], grad_fn=<AddBackward0>)

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            print(model)
            model.send(data.location)
            print(model)
            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0038, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0033, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0026, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0023, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0019, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0016, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.0013, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2

In [0]:
model(torch.tensor([1,1.],requires_grad=True))

tensor([1.0562], grad_fn=<AddBackward0>)

In [0]:
params = list(model.parameters())

In [0]:
params[0].grad

tensor([[-0.0640,  0.1084]])

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
bob._objects

{}

## SPLIT NN ON TOY DATASET 

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0,0],[0,0,1],[1,1,0],[1,0,1.],[1,1,1],[1,0,0],[0,1,0],[0,1,1]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.],[1],[1],[0],[0]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:4]
target_bob = target[0:4]

data_alice = data[4:]
target_alice = target[4:]

# Iniitalize A Toy Model
model = nn.Sequential(
    nn.Linear(3,2),
    nn.ReLU(),
    nn.Linear(2,1)
)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            print(model)
            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)
            #pr = model(data).get() 
            #print("prediction:",pr)
            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()
            
            print(list(model.parameters())[1].grad)

            # 6) print our progress
            print("loss:",loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([4.0322, 1.2717])
loss: tensor(3.9985, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.4976, -0.3512])
loss: tensor(1.1500, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([ 0.1877, -0.2439])
loss: tensor(0.9486, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.3734, -0.1508])
loss: tensor(0.8084, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.0019, -0.2647])
los

In [0]:
model(torch.tensor([0,0.,0.],requires_grad=True))

tensor([0.0737], grad_fn=<AddBackward0>)

In [0]:
for data,target in datasets:

    # NEW) send model to correct worker
    model.send(data.location)

    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # NEW) get model (with gradients)
    model.get()
    print(list(model.parameters())[1].grad)
    # 6) print our progress
    print('loss:',loss.get()) # NEW) slight edit... need to call .get() on loss\

tensor([-3.5763, -3.5968])
loss: tensor(3.7525, requires_grad=True)
tensor([3.2863, 3.2910])
loss: tensor(4.0966, requires_grad=True)


In [0]:
import copy

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0,0],[0,0,1],[1,1,0],[1,0,1.],[1,1,1],[1,0,0],[0,1,0],[0,1,1]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.],[1],[1],[0],[0]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:4]
target_bob = target[0:4]

data_alice = data[4:]
target_alice = target[4:]

# Iniitalize A Toy Model
# one model per client 
models = [nn.Sequential(nn.Linear(3,2),nn.ReLU()),nn.Sequential(nn.Linear(3,2),nn.ReLU())]
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)

### MODEL A, can keep the labels on server 
#target_bob = target_bob.send(bob)
#target_alice = target_alice.send(alice)

#send models ti the clients 
models[0] = models[0].send(bob)
models[1] = models[1].send(alice)

opt1 = optim.SGD(params=models[0].parameters(),lr=0.1)
opt2 = optim.SGD(params=models[1].parameters(),lr=0.1)
# organize pointers into a list
datasets = [(models[0],data_bob,target_bob,opt1),(models[1],data_alice,target_alice,opt2)]


opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    #opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):
        
        # NEW) iterate through each worker's dataset
        for mod_c,data,target,opt_c in datasets:
            
            # 1) erase previous gradients (if they exist)
            opt.step()
            opt_c.step()
            opt_c.zero_grad()
            opt.zero_grad()

            # 2) make a prediction until cut layer (client location)
            pred_c = mod_c(data)
            copy = pred_c.copy()
            # 3) get this to the server 
            inp = copy.get()
            
            # 4) make prediction with second part of the model (server location)
            pred = model(inp)
            
            # 5) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 6) figure out which weights caused us to miss
            loss.backward()

            # 7) change those weights
            #opt.step()
            
            # 8) send the last gradient to the client 
            gradient = inp.grad
            
            #gradient = list(model.parameters())[0].grad
            #print("gradient=",gradient)
            gradient = gradient.send(data.location)
            #print("gradient=",gradient)
            
            # 9) figure out which weights caused us to miss with that gradient as input 
            #pred_c.requires_grad_(True)
            #pred_c.grad = gradient
            
            #mod_c.get()
            #print(list(mod_c.parameters())[-1].requires_grad)
            #list(mod_c.parameters())[-1].grad = gradient
            #print(list(mod_c.parameters())[-1].grad)
            #mod_c = mod_c.send(data.location)
            #pred_c.backward(torch.ones(pred_c.size()),retain_graph=True)
            #print(pred_c)
            pred_c.backward(gradient)
            
            # 10) change those weights
            #opt.step()
            #opt_c.step()
  
            #params = list(mod_c.get().parameters())
            #params = list(model.parameters())
            #print("params",params[0])
            #mod_c.send(data.location)
            #print('data1',data)
            #D = data.copy()
            #loc = D.location
            #print("grad:", data.grad.clone().get())
            #print("data1,5",data)
            #data = data.send(loc)
            #print("data2",data)
        
            # 11) print our progress
            print("loss:",loss) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

loss: tensor(0.0004, grad_fn=<SumBackward0>)
loss: tensor(0.0151, grad_fn=<SumBackward0>)
loss: tensor(0.0004, grad_fn=<SumBackward0>)
loss: tensor(0.0113, grad_fn=<SumBackward0>)
loss: tensor(0.0004, grad_fn=<SumBackward0>)
loss: tensor(0.0085, grad_fn=<SumBackward0>)
loss: tensor(0.0002, grad_fn=<SumBackward0>)
loss: tensor(0.0063, grad_fn=<SumBackward0>)
loss: tensor(0.0001, grad_fn=<SumBackward0>)
loss: tensor(0.0048, grad_fn=<SumBackward0>)
loss: tensor(0.0001, grad_fn=<SumBackward0>)
loss: tensor(0.0036, grad_fn=<SumBackward0>)
loss: tensor(0.0001, grad_fn=<SumBackward0>)
loss: tensor(0.0026, grad_fn=<SumBackward0>)
loss: tensor(0.0001, grad_fn=<SumBackward0>)
loss: tensor(0.0019, grad_fn=<SumBackward0>)
loss: tensor(4.1138e-05, grad_fn=<SumBackward0>)
loss: tensor(0.0014, grad_fn=<SumBackward0>)
loss: tensor(2.9122e-05, grad_fn=<SumBackward0>)
loss: tensor(0.0011, grad_fn=<SumBackward0>)
loss: tensor(5.6937e-05, grad_fn=<SumBackward0>)
loss: tensor(0.0008, grad_fn=<SumBackward0>

In [0]:
model(models[0].get()(torch.tensor([0,0.,1.])))

tensor([-0.0005], grad_fn=<AddBackward0>)

In [0]:
model(models[0].get()(torch.tensor([0,0.,0.])))

tensor([0.0084], grad_fn=<AddBackward0>)

In [0]:
models[0] = models[0].send(bob)

In [0]:
model(models[1].get()(torch.tensor([0,0.,1.])))

tensor([0.3594], grad_fn=<AddBackward0>)

In [0]:
models[1] = models[1].send(alice)

In [0]:
model(models[1].get()(torch.tensor([1,1.,1.])))

tensor([1.0007], grad_fn=<AddBackward0>)

In [0]:
model(models[0].get()(torch.tensor([1,1.,1.])))

tensor([0.9635], grad_fn=<AddBackward0>)

In [0]:
models[0] = models[0].send(bob)

In [0]:
i = 0 
for p in model.parameters():
  print(i)
  if (i==0):
    p.grad = torch.tensor([1.,2.],requires_grad=True)
  print(p)
  i+=1
  
  
for p in model.parameters():
  print(i)
  print(p.grad)
  i+=1

0
Parameter containing:
tensor([[3., 3.]], grad_fn=<AddBackward0>)
1
Parameter containing:
tensor([0.4538], requires_grad=True)
2
tensor([1., 2.], grad_fn=<NotImplemented>)
3
tensor([0.])


In [0]:
models[0]

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
models[0].get()

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
list(models[0].parameters())[-1].grad =  torch.tensor([4.,3.])

In [0]:
print(list(models[0].parameters())[-1].grad)

tensor([4., 3.])


In [0]:
list(models[0].parameters())[-1].grad 

tensor([4., 3.])

In [0]:
a = torch.tensor([1,1,1,1.],requires_grad=True)

In [0]:
a

tensor([1., 1., 1., 1.], requires_grad=True)

In [0]:
b = a + 2 

In [0]:
b

tensor([3., 3., 3., 3.], grad_fn=<AddBackward0>)

In [0]:
b.backward(torch.tensor([1.,1.,1.,1.]))

In [0]:
print(b.grad)

None


In [0]:
print(a.grad)

tensor([3., 3., 3., 3.])


In [0]:
a.grad.zero_()

tensor([0., 0., 0., 0.])

In [0]:
torch.ones(3)

tensor([1., 1., 1.])

## TRUSTED AGGREGATOR

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0697) Alice:tensor(0.0006)
Bob:tensor(0.0508) Alice:tensor(4.2326e-05)
Bob:tensor(0.0376) Alice:tensor(0.0002)
Bob:tensor(0.0281) Alice:tensor(0.0002)
Bob:tensor(0.0211) Alice:tensor(0.0002)
Bob:tensor(0.0160) Alice:tensor(0.0002)
Bob:tensor(0.0121) Alice:tensor(0.0002)
Bob:tensor(0.0092) Alice:tensor(0.0001)
Bob:tensor(0.0071) Alice:tensor(0.0001)
Bob:tensor(0.0054) Alice:tensor(9.1933e-05)


## Federated Learning on MNIST using a CNN

In [0]:
! pip install syft

     |████████████████████████████████| 153kB 30.7MB/s 
     |████████████████████████████████| 204kB 35.2MB/s 
     |████████████████████████████████| 450kB 27.7MB/s 
     |████████████████████████████████| 71kB 30.8MB/s 
     |████████████████████████████████| 560.1MB 28kB/s 
     |████████████████████████████████| 368kB 44.2MB/s 
     |████████████████████████████████| 51kB 19.7MB/s 
     |████████████████████████████████| 122kB 51.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
Successfully built zstd
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
device

device(type='cpu')

In [2]:
D = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

0it [00:00, ?it/s]

9920512it [00:01, 8243483.64it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135517.18it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2222132.02it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51890.78it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [9]:
D[0][0].size()

torch.Size([1, 28, 28])

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

RuntimeError: ignored

## Tests

In [0]:
x = torch.tensor([1.,2.,1.,3.])

In [0]:
hey = sy.VirtualWorker(hook,id="hey")

In [0]:
hey._objects

{}

In [0]:
x

tensor([1., 2., 1., 3.])

In [0]:
x.send(hey)

(Wrapper)>[PointerTensor | me:13407371945 -> hey:56027152132]

In [0]:
x

tensor([1., 2., 1., 3.])

In [0]:
x.get()

AttributeError: ignored

In [0]:
hey._objects

{56027152132: tensor([1., 2., 1., 3.])}

In [0]:
x = x.send(hey)

In [0]:
x

(Wrapper)>[PointerTensor | me:88772131240 -> hey:56027152132]

In [0]:
hey._objects

{56027152132: tensor([1., 2., 1., 3.])}

In [0]:
x=x.get()

In [0]:
x

tensor([1., 2., 1., 3.])

In [0]:
ml = nn.Sequential(nn.Linear(3,2),nn.ReLU())

In [0]:
ml

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
ml = ml.send(hey)

In [0]:
ml

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
hey._objects

{44922572202: Parameter containing:
 tensor([[ 0.4500,  0.0126, -0.2453],
         [-0.3346,  0.0929, -0.0680]], requires_grad=True),
 81399605280: Parameter containing:
 tensor([-0.1135, -0.0340], requires_grad=True)}

In [0]:
ml = ml.get()

In [0]:
hey._objects

{}

In [0]:
ml=ml.send(bob)

In [0]:
ml = ml.move(hey)

In [0]:
hey._objects

{110764493: Parameter containing:
 tensor([-0.1135, -0.0340], requires_grad=True),
 44922572202: Parameter containing:
 tensor([[ 0.4500,  0.0126, -0.2453],
         [-0.3346,  0.0929, -0.0680]], requires_grad=True),
 66803002729: Parameter containing:
 tensor([[ 0.4500,  0.0126, -0.2453],
         [-0.3346,  0.0929, -0.0680]], requires_grad=True),
 81399605280: Parameter containing:
 tensor([-0.1135, -0.0340], requires_grad=True)}

In [0]:
print(ml)

None


In [0]:
test = sy.VirtualWorker(hook,id="test")

In [0]:
ml = nn.Sequential(nn.Linear(3,2),nn.ReLU())

In [0]:
ml = ml.send(hey)

In [0]:
ml.move(test)

In [0]:
test._objects

{2655579637: Parameter containing:
 tensor([[-0.4470, -0.4307, -0.0440],
         [ 0.0981,  0.0169, -0.3907]], requires_grad=True),
 26877253094: Parameter containing:
 tensor([ 0.3501, -0.0431], requires_grad=True),
 40177035808: Parameter containing:
 tensor([[-0.4470, -0.4307, -0.0440],
         [ 0.0981,  0.0169, -0.3907]], requires_grad=True),
 56675706551: Parameter containing:
 tensor([ 0.3501, -0.0431], requires_grad=True)}

In [0]:
ml = ml.get()

In [0]:
ml

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
test._objects

{26877253094: Parameter containing:
 tensor([ 0.3501, -0.0431], requires_grad=True),
 40177035808: Parameter containing:
 tensor([[-0.4470, -0.4307, -0.0440],
         [ 0.0981,  0.0169, -0.3907]], requires_grad=True)}

In [0]:
list(ml.parameters())[0]

Parameter containing:
tensor([[-0.4470, -0.4307, -0.0440],
        [ 0.0981,  0.0169, -0.3907]], requires_grad=True)

In [0]:
list(ml.parameters())[1]

Parameter containing:
tensor([ 0.3501, -0.0431], requires_grad=True)

In [0]:
M

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
)

In [0]:
M = M.send(hey)

In [0]:
x = torch.tensor([1.,6.])

In [0]:
x = x.send(hey)

In [0]:
M = nn.Linear(2,1)

In [0]:
list(M.parameters())

[Parameter containing:
 tensor([[ 0.6992, -0.5301]], requires_grad=True), Parameter containing:
 tensor([0.5731], requires_grad=True)]

In [0]:
list(M.parameters())[1].grad = x

In [0]:
print(list(M.parameters())[1].grad)

None


In [0]:
M = M.send(test)

In [0]:
x = torch.tensor([1.,2.])

In [0]:
x = x.send(test)

In [0]:
M.weight.data.set_(x)

(Wrapper)>[PointerTensor | me:45097543585 -> test:12798246557]::data

In [0]:
list(M.parameters())[0].get()

Parameter containing:
tensor([[ 0.6992, -0.5301]], requires_grad=True)

In [0]:
net1 = nn.Sequential(nn.Linear(3,2), nn.ReLU())
net2 = nn.Linear(2,1)

In [0]:
data = torch.tensor([[0,0,0],[0,0,1],[1,1,0],[1,0,1.],[1,1,1],[1,0,0],[0,1,0],[0,1,1]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.],[1],[1],[0],[0]], requires_grad=True)

opt1 = optim.SGD(params=net1.parameters(),lr=0.1)
opt2 = optim.SGD(params=net2.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt1.zero_grad()
        opt2.zero_grad()
        
        # 2) make a prediction
        out1 = net1(data)
        pred = net2(out1)
        #print(pred)
        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt1.step()
        opt2.step()

        # 6) print our progress
        print(loss.data)


In [0]:
train()

tensor(8.8212)
tensor(54.3115)
tensor(120.3988)
tensor(44.6236)
tensor(17.3445)
tensor(7.5240)
tensor(3.9886)
tensor(2.7159)
tensor(2.2577)
tensor(2.0928)
tensor(2.0334)
tensor(2.0120)
tensor(2.0043)
tensor(2.0016)
tensor(2.0006)
tensor(2.0002)
tensor(2.0001)
tensor(2.0000)
tensor(2.0000)
tensor(2.0000)


In [0]:
out1 = net1(data)

In [0]:
out1

tensor([[0.0000, 0.2896],
        [0.0695, 0.0939],
        [0.0000, 0.2404],
        [0.0000, 0.0000],
        [0.0000, 0.0446],
        [0.0000, 0.1428],
        [0.0000, 0.3872],
        [0.0000, 0.1915]], grad_fn=<ThresholdBackward0>)

In [0]:
out2 = net2(out1)

In [0]:
out2

tensor([[-0.5381],
        [-0.6299],
        [-0.5632],
        [-0.6858],
        [-0.6630],
        [-0.6130],
        [-0.4883],
        [-0.5881]], grad_fn=<AddmmBackward>)

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
#model = nn.Sequential(nn.Linear(2,3),nn.Linear(3,1))
net1 = nn.Linear(2,3)
net2 = nn.Linear(3,1)

def train():
    # Training Logic
    opt1 = optim.SGD(params=net1.parameters(),lr=0.1)
    opt2 = optim.SGD(params=net2.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt1.zero_grad()
        opt2.zero_grad()
        
        # 2) make a prediction
        out1 = net1(data)
        pred = net2(out1)
        #print(pred)
        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) change those weights
        opt1.step()
        opt2.step()
        # 6) print our progress
        print(loss.data)
        

In [0]:
train()

tensor(0.9131)
tensor(0.7583)
tensor(0.6816)
tensor(0.6703)
tensor(0.8395)
tensor(1.1019)
tensor(1.8191)
tensor(1.5124)
tensor(1.4827)
tensor(0.4298)
tensor(0.2546)
tensor(0.1483)
tensor(0.1033)
tensor(0.0740)
tensor(0.0557)
tensor(0.0423)
tensor(0.0329)
tensor(0.0256)
tensor(0.0206)
tensor(0.0167)


In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
#model = nn.Sequential(nn.Linear(2,3),nn.Linear(3,1))
net1 = nn.Linear(2,3)
net2 = nn.Linear(3,1)


opt1 = optim.SGD(params=net1.parameters(),lr=0.1)
opt2 = optim.SGD(params=net2.parameters(),lr=0.1)


for iter in range(20):

    # 1) erase previous gradients (if they exist)
    opt1.zero_grad()
    opt2.zero_grad()

    # 2) make a prediction
    out1 = net1(data)
    #out1.register_hook(print)
    pred = net2(out1)
    #print(pred)
    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()


    # 5) change those weights
    opt1.step()
    opt2.step()
    # 6) print our progress
    print(loss.data)
        

tensor(6.3278)
tensor(0.6669)
tensor(0.3819)
tensor(0.2574)
tensor(0.1789)
tensor(0.1241)
tensor(0.0858)
tensor(0.0592)
tensor(0.0406)
tensor(0.0278)
tensor(0.0189)
tensor(0.0128)
tensor(0.0086)
tensor(0.0058)
tensor(0.0038)
tensor(0.0026)
tensor(0.0017)
tensor(0.0011)
tensor(0.0007)
tensor(0.0005)


In [0]:
x.grad.zero_()

tensor([0., 0.])

In [0]:
x = torch.tensor([3.,4.],requires_grad=True)

In [0]:
y = x**2


In [0]:
y.register_hook(print)

In [0]:
z = 3*y[0]+2*y[1]

In [0]:
z.backward()

tensor([3., 2.])


In [0]:
z.grad

In [0]:
y.grad

In [0]:
x.grad

tensor([6., 8.])

In [0]:
y.backward(torch.ones(y.size()))

In [0]:
a = torch.tensor([1.,1.],requires_grad=True)

In [0]:
b = a**2

In [0]:
b.register_hook(print)

In [0]:
c = b[0] + 3*b[1]

In [0]:
c.backward()

tensor([1., 3.])


In [0]:
a.grad

tensor([2., 6.])

In [0]:
b.backward(torch.tensor([1., 3.]))

In [0]:
a.grad

tensor([2., 6.])

In [0]:
q = torch.tensor([1.,1.],requires_grad=True)

In [0]:
p = q**2

In [0]:
p.backward(torch.tensor([1.,3.]))

In [0]:
q.grad

tensor([2., 6.])

In [0]:
x = torch.tensor([2.,2.])

In [0]:
x = x.send(bob)

In [0]:
y = x**2

In [0]:
z = y.copy()

In [0]:
tmp = y.get()

In [0]:
y

tensor([])

In [0]:
tmp

tensor([4., 4.])

In [0]:
x

(Wrapper)>[PointerTensor | me:76251588823 -> bob:87570781890]

In [0]:
z

(Wrapper)>[PointerTensor | me:74590764005 -> bob:14538934924]

# Split NN configuration B

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0,0],[0,0,1],[1,1,0],[1,0,1.],[1,1,1],[1,0,0],[0,1,0],[0,1,1]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.],[1],[1],[0],[0]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:4]
target_bob = target[0:4]

data_alice = data[4:]
target_alice = target[4:]

# Iniitalize A Toy Model
model = nn.Sequential(
    nn.Linear(3,2),
    nn.ReLU(),
    nn.Linear(2,1)
)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            print(model)
            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)
            #pr = model(data).get() 
            #print("prediction:",pr)
            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()
            
            print(list(model.parameters())[1].grad)

            # 6) print our progress
            print("loss:",loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([4.0322, 1.2717])
loss: tensor(3.9985, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.4976, -0.3512])
loss: tensor(1.1500, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([ 0.1877, -0.2439])
loss: tensor(0.9486, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.3734, -0.1508])
loss: tensor(0.8084, requires_grad=True)
Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
)
tensor([-0.0019, -0.2647])
los